In [1]:
import pandas as pd

compositional_preset = pd.read_csv('data/Compositional Preset.csv')
answer = pd.read_csv('data/english_learning_proud_moments.csv')

In [2]:
compositional_preset['TOPICS'][15]

'1. Describe a time when you felt proud of yourself and explain why.\n2. Share a personal achievement that made you feel proud and describe the steps you took to accomplish it.\n3. Discuss a time when someone else made you feel proud and explain how their actions or accomplishments impacted you.'

In [3]:
answer

,Level,Proud Moment,Personal Achievement,Others' Impact
0,A1 (Beginner),I felt proud when I speak English first time. ...,I learn English. I listen and repeat. I proud.,My friend say thank you to me. I help her. I f...
1,A2 (Elementary),I felt proud when I ordered food in English. I...,I finished a big puzzle. I did a little every ...,My brother won a game. He practiced a lot. I a...
2,B1 (Intermediate),I was proud when I gave a talk in English clas...,I ran a 10k race. I trained for months and imp...,My teacher got an award. She teaches so well. ...
3,B2 (Upper Intermediate),I felt immensely proud when I wrote an essay i...,Completing a half-marathon was a triumph. The ...,Seeing my friend volunteer and help the commun...
4,C1 (Advanced),A moment of profound pride for me was when I p...,Publishing my first research paper was a landm...,Witnessing my colleague's advocacy in environm...


# Comprehension

## cosine_similarity

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [5]:
# Load the provided CSV file into a DataFrame
answers_csv_path = 'data/english_learning_proud_moments.csv'
answers_df = pd.read_csv(answers_csv_path)

# Given question string
question = ("1. Describe a time when you felt proud of yourself and explain why.\n"
            "2. Share a personal achievement that made you feel proud and describe the steps you took to accomplish it.\n"
            "3. Discuss a time when someone else made you feel proud and explain how their actions or accomplishments impacted you.")

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Concatenate the question with each answer to form a corpus for each column
corpora = {
    column: [question] + answers_df[column].tolist() for column in answers_df.columns if column != 'Level'
}

# Calculate cosine similarity between the question and each answer in the DataFrame
similarity_scores = {}

for column, corpus in corpora.items():
    # Vectorize the corpus
    tfidf_matrix = vectorizer.fit_transform(corpus)
    
    # Calculate cosine similarity between the question and each answer
    # The question is the first document in the corpus, so it's index 0
    # We compare it with all subsequent documents (the answers)
    cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:])
    
    # Add the scores to the dictionary with the corresponding level labels
    similarity_scores[column] = dict(zip(answers_df['Level'], cosine_similarities.flatten()))

similarity_scores_df = pd.DataFrame(similarity_scores)
similarity_scores_df


,Proud Moment,Personal Achievement,Others' Impact
A1 (Beginner),0.160201,0.119145,0.267306
A2 (Elementary),0.107328,0.029455,0.059247
B1 (Intermediate),0.100734,0.194318,0.092337
B2 (Upper Intermediate),0.083293,0.086658,0.194780
C1 (Advanced),0.108909,0.086823,0.007107


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the provided CSV file into a DataFrame
answers_csv_path = 'data/english_learning_proud_moments.csv'
answers_df = pd.read_csv(answers_csv_path)

# Split the given question string into individual questions
questions = [
    "Describe a time when you felt proud of yourself and explain why.",
    "Share a personal achievement that made you feel proud and describe the steps you took to accomplish it.",
    "Discuss a time when someone else made you feel proud and explain how their actions or accomplishments impacted you."
]

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Function to calculate cosine similarity for each question with the corresponding column in the DataFrame
def calculate_similarity(question, answers):
    # Vectorize the question and answers
    tfidf_matrix = vectorizer.fit_transform([question] + answers)
    
    # Calculate cosine similarity between the question and each answer
    # The question is the first document, so compare it with the answers
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    
    # Flatten to get a list of similarities
    return cosine_similarities.flatten()

# Calculate cosine similarity for each question
similarity_scores = {
    'Proud Moment': calculate_similarity(questions[0], answers_df['Proud Moment'].tolist()),
    'Personal Achievement': calculate_similarity(questions[1], answers_df['Personal Achievement'].tolist()),
    "Others' Impact": calculate_similarity(questions[2], answers_df["Others' Impact"].tolist())
}

# Create a DataFrame to display the similarity scores
similarity_scores_df = pd.DataFrame(similarity_scores, index=answers_df['Level'])
similarity_scores_df

,Proud Moment,Personal Achievement,Others' Impact
Level,,,
A1 (Beginner),0.234307,0.087214,0.205707
A2 (Elementary),0.101725,0.064683,0.020885
B1 (Intermediate),0.083160,0.157087,0.050160
B2 (Upper Intermediate),0.102304,0.154683,0.113646
C1 (Advanced),0.154355,0.093320,0.000000
